In [20]:
import pickle
from datasets import load_dataset
import random

# KorQuAD 데이터셋 로드
dataset = load_dataset("KorQuAD/squad_kor_v1")

# 상위 10개 샘플만 선택
subset = dataset['train'].select(range(10))

# 데이터 준비 (상위 10개만 선택)
questions = [sample['question'] for sample in subset]
contexts = [sample['context'] for sample in subset]
answers = [sample['answers']['text'][0] for sample in subset]

# 각 질문에 대해 여러 개의 Context 생성 (정답 포함 + 무작위 Context 추가)
num_contexts = 3  # 각 질문에 대해 3개의 Context 설정
all_contexts = []
for i in range(len(questions)):
    context_list = [contexts[i]]  # 정답이 포함된 실제 Context
    for _ in range(num_contexts - 1):  # 나머지 Context는 무작위로 추가
        random_context = random.choice(contexts)
        context_list.append(random_context)
    all_contexts.append(context_list)

# RAG 데이터 구조 생성
rag_data = {
    'questions': questions,
    'contexts': all_contexts,
    'contexts_answer_idx': [0] * len(questions),  # 실제 답변이 포함된 Context 인덱스를 첫 번째로 설정
    'answers': answers
}

# pickle로 데이터 저장
with open('korquad_rag_data_10.pkl', 'wb') as f:
    pickle.dump(rag_data, f)


In [21]:
len(questions)

10

In [22]:
for question in questions[:5]:
    print(question)

바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?
바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?
바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?
1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?
파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?


In [ ]:
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))



In [24]:
from tqdm import tqdm
from utils import call_openai, get_embeddings, cosine_similarity

predictions = []

# 각 질문에 대해 OpenAI 모델을 호출하여 예측 응답을 생성
for i in tqdm(range(len(questions))):
    prompt = f"""You are an expert in reading comprehension and Korean general knowledge. Please answer to the question given below. Use information given in Context appropriately.
    
Question:
{questions[i]}
"""

    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')  # OpenAI 모델 호출하여 응답 생성
    predictions.append(prediction)  # 생성된 응답을 predictions 리스트에 추가


100%|██████████| 10/10 [00:18<00:00,  1.88s/it]


In [25]:
predictions

['바그너는 괴테의 "파우스트"를 읽고 "파우스트 서곡"을 쓰고자 했습니다.',
 '바그너는 교향곡 작곡을 제1악장까지 쓴 뒤에 중단했습니다.',
 '바그너가 파우스트 서곡을 쓸 때 베를리오즈의 "환상 교향곡"의 영향을 받았습니다.',
 '1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은 "파우스트"입니다.',
 "질문에 답하기 위해서는 파우스트 서곡의 라단조 조성에 영향을 준 베토벤의 곡을 알아야 합니다. 파우스트 서곡은 독일 작곡가 리하르트 바그너가 작곡한 곡으로, 베토벤의 영향을 많이 받았습니다. 특히, 베토벤의 교향곡 9번 '합창'의 1악장이 라단조로 되어 있어 파우스트 서곡의 라단조 조성에 영향을 준 것으로 알려져 있습니다.\n\n따라서, 파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은 **교향곡 9번 '합창'**입니다.",
 '바그너가 파우스트를 처음으로 읽은 년도는 1831년입니다.',
 '질문에 답하기 위해서는 바그너가 처음 교향곡을 작곡한 장소에 대한 정보가 필요합니다. 주어진 문맥(Context)에서 이 정보를 찾아야 합니다. 그러나 현재 제공된 문맥이 없기 때문에, 일반적인 지식을 바탕으로 답변을 드리겠습니다.\n\n리하르트 바그너(Richard Wagner)는 주로 오페라 작곡가로 알려져 있으며, 그의 교향곡 작곡 활동은 상대적으로 덜 알려져 있습니다. 바그너는 1832년에 첫 번째 교향곡을 작곡했으며, 이 교향곡은 독일의 라이프치히에서 작곡되었습니다.\n\n따라서, 바그너가 처음 교향곡을 작곡한 장소는 **독일 라이프치히**입니다.',
 '질문에 답하기 위해서는 바그너의 1악장이 초연된 장소에 대한 구체적인 정보가 필요합니다. 그러나 주어진 문맥(Context)에서 이 정보를 제공하지 않았기 때문에 정확한 답변을 드릴 수 없습니다. 바그너의 1악장이 초연된 장소에 대한 정보를 제공해 주시면, 그에 따라 답변을 드릴 수 있습니다.',
 '질문에 답하기 위해서는 주어진 문맥(Context)이 필요합니다. 주어진 문맥이 없기 때문

In [27]:
from ragas.metrics import FactualCorrectness
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset
from ragas import evaluate

# 평가할 샘플 생성
samples = []
for question, ai_response, ground_truth in zip(questions, predictions, answers):
    sample = SingleTurnSample(
        user_input=question,       # 사용자 질문
        response=ai_response,      # AI의 응답
        reference=ground_truth     # 정답
    )
    samples.append(sample)

# 평가 데이터셋 생성
dataset = EvaluationDataset(samples)

# Factual Correctness 메트릭 정의 (llm 매개변수 제거)
factual_correctness_metric = FactualCorrectness()

# 평가 수행
results = evaluate(dataset, metrics=[factual_correctness_metric])

# 결과를 데이터프레임으로 변환하여 출력
results_df = results.to_pandas()
print(results_df.head())


Evaluating: 100%|██████████| 10/10 [00:14<00:00,  1.40s/it]


                          user_input  \
0     바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?   
1      바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?   
2  바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?   
3     1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?   
4   파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?   

                                            response    reference  \
0           바그너는 괴테의 "파우스트"를 읽고 "파우스트 서곡"을 쓰고자 했습니다.          교향곡   
1                   바그너는 교향곡 작곡을 제1악장까지 쓴 뒤에 중단했습니다.          1악장   
2      바그너가 파우스트 서곡을 쓸 때 베를리오즈의 "환상 교향곡"의 영향을 받았습니다.  베토벤의 교향곡 9번   
3           1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은 "파우스트"입니다.         파우스트   
4  질문에 답하기 위해서는 파우스트 서곡의 라단조 조성에 영향을 준 베토벤의 곡을 알아...        합창교향곡   

   factual_correctness  
0                 0.00  
1                 0.00  
2                 0.00  
3                 0.67  
4                 0.00  


In [28]:
for i, response in enumerate(results_df['response']):
    print(f"Response {i+1}:")
    print(response)
    print("-" * 80)  # 구분선


Response 1:
바그너는 괴테의 "파우스트"를 읽고 "파우스트 서곡"을 쓰고자 했습니다.
--------------------------------------------------------------------------------
Response 2:
바그너는 교향곡 작곡을 제1악장까지 쓴 뒤에 중단했습니다.
--------------------------------------------------------------------------------
Response 3:
바그너가 파우스트 서곡을 쓸 때 베를리오즈의 "환상 교향곡"의 영향을 받았습니다.
--------------------------------------------------------------------------------
Response 4:
1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은 "파우스트"입니다.
--------------------------------------------------------------------------------
Response 5:
질문에 답하기 위해서는 파우스트 서곡의 라단조 조성에 영향을 준 베토벤의 곡을 알아야 합니다. 파우스트 서곡은 독일 작곡가 리하르트 바그너가 작곡한 곡으로, 베토벤의 영향을 많이 받았습니다. 특히, 베토벤의 교향곡 9번 '합창'의 1악장이 라단조로 되어 있어 파우스트 서곡의 라단조 조성에 영향을 준 것으로 알려져 있습니다.

따라서, 파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은 **교향곡 9번 '합창'**입니다.
--------------------------------------------------------------------------------
Response 6:
바그너가 파우스트를 처음으로 읽은 년도는 1831년입니다.
--------------------------------------------------------------------------------
Response

In [31]:
print(results)

{'factual_correctness': 0.2150}


In [32]:
print(questions[0])

바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?


In [33]:
print(predictions[0])

바그너는 괴테의 "파우스트"를 읽고 "파우스트 서곡"을 쓰고자 했습니다.


In [34]:
import numpy as np
from tqdm import tqdm
from utils import call_openai, get_embeddings, cosine_similarity

# 질문에 대해 가장 유사한 Context를 검색하는 함수
def retrieve_context(question, contexts):
    question_embedding = get_embeddings([question], model='text-embedding-3-large')[0]  # 질문의 임베딩 생성
    context_embeddings = get_embeddings(contexts, model='text-embedding-3-large')  # Context 임베딩 생성

    # 질문과 각 Context 간의 유사도 계산
    similarities = [cosine_similarity(question_embedding, context_embedding) for context_embedding in context_embeddings]

    # 유사도가 가장 높은 Context 인덱스 반환
    most_relevant_index = np.argmax(similarities)
    return contexts[most_relevant_index]

# 각 질문에 대해 가장 관련 있는 Context를 사용하여 OpenAI 모델에 질문
predictions = []
for i in tqdm(range(len(questions))):
    context = retrieve_context(questions[i], contexts[i])  # 가장 관련 있는 Context 검색
    prompt = f"""You are an expert in reading comprehension and Korean general knowledge. Please answer to the question given below. Use information given in Context appropriately.

Context:
{context}

Question:
{questions[i]}
"""
    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')  # 모델에 질문과 Context로 응답 생성
    predictions.append(prediction)  # 생성된 응답을 predictions 리스트에 추가


100%|██████████| 10/10 [00:32<00:00,  3.21s/it]


In [37]:
from ragas.metrics import FactualCorrectness
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset
from ragas import evaluate

# 각 질문, AI 응답, 정답을 사용해 평가할 샘플 생성
samples = []
for question, ai_response, ground_truth in zip(questions, predictions, answers):
    sample = SingleTurnSample(
        user_input=question,       # 사용자 질문
        response=ai_response,      # AI의 응답
        reference=ground_truth     # 정답
    )
    samples.append(sample)

# 평가 데이터셋 생성
dataset = EvaluationDataset(samples)

# Factual Correctness 메트릭 설정
factual_correctness_metric = FactualCorrectness()

# 평가 수행 및 결과 저장
results = evaluate(dataset, metrics=[factual_correctness_metric])

# 결과를 데이터프레임으로 변환하여 출력
results_df = results.to_pandas()
print(results_df.head())


Evaluating: 100%|██████████| 10/10 [00:09<00:00,  1.08it/s]


                          user_input  \
0     바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?   
1      바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?   
2  바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?   
3     1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?   
4   파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?   

                                            response    reference  \
0                   바그너는 괴테의 파우스트를 읽고 오페라를 쓰고자 했습니다.          교향곡   
1                   바그너는 교향곡 작곡을 제1악장까지 쓴 뒤에 중단했습니다.          1악장   
2      바그너가 파우스트 서곡을 쓸 때 베를리오즈의 "환상 교향곡"의 영향을 받았습니다.  베토벤의 교향곡 9번   
3           1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은 "파우스트"입니다.         파우스트   
4  파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은 "교향곡 5번"입니다. 베...        합창교향곡   

   factual_correctness  
0                 0.00  
1                 0.00  
2                 0.00  
3                 0.67  
4                 0.00  


In [30]:
from sentence_transformers import SentenceTransformer, util

# Sentence-BERT 모델을 사용한 임베딩 유사도 계산
model = SentenceTransformer('all-MiniLM-L6-v2')
similarity_scores = []

for question, ai_response, ground_truth in zip(questions, predictions, answers):
    # 임베딩 생성
    embedding_ref = model.encode(ground_truth, convert_to_tensor=True)
    embedding_res = model.encode(ai_response, convert_to_tensor=True)
    
    # 코사인 유사도 계산
    similarity = util.pytorch_cos_sim(embedding_ref, embedding_res).item()
    similarity_scores.append(similarity)

# 유사도 점수 출력
for score in similarity_scores:
    print(f"유사도 점수: {score:.2f}")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
c:\Users\Byung Mu Kang\RAG-1stWeek\myenv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Byung Mu Kang\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In 

유사도 점수: 0.29
유사도 점수: 0.57
유사도 점수: 0.36
유사도 점수: 0.75
유사도 점수: 0.06
유사도 점수: 0.17
유사도 점수: 0.13
유사도 점수: 0.14
유사도 점수: 0.12
유사도 점수: 0.31
